In [1]:
from pyforestscan.calculate import calculate_chm, calculate_fhd, assign_voxels
from pyforestscan.handlers import read_lidar, create_geotiff
import numpy as np
import glob, os

In [2]:
path = "/home/mgallet/Documents/DATA/LIDAR_TEST/processed/lidar_data/Belledonne/**/*.npy"
path2 = "/home/mgallet/Documents/DATA/LIDAR_TEST/processed_V2_*/**/*.npy"

In [ ]:
import pdal,json

pipe = pdal.Pipeline(json.dumps({
    "pipeline": [
        {
            "type": "readers.las",
            "filename": "/home/mgallet/Téléchargements/uncompress_LHD_FXX_0945_6473_PTS_O_LAMB93_IGN69.las",
            "spatialreference": "EPSG:2154",
            "start": 0,
            "count": 50000
        },
        {
        "type": "filters.voxeldownsize",
        "cell": 0.1,
        },
      {
        "type": "filters.outlier",
        "method": "statistical",
        "mean_k": 20,
        "multiplier": 2
      },
      {
        "type": "filters.elm",
        "cell": 2,
        "class": 2,
        "threshold": 0.5
      },
      # {
      #   "type": "filters.assign",
      #   "assignment": "Classification[2:2]=0"
      # },
      {
        "type": "filters.csf",
        "resolution": 0.375,
        "rigidness": 3,
        "iterations": 500,
        "threshold": 0.15,
        "step": 0.75,
        "ignore": "Classification[7:7]",
      },
     
     {
      "type": "filters.csf",
      "resolution": 0.5,
      "rigidness": 3,
      "iterations": 250,
      "threshold": 0.2,
  },
  {
      "type": "filters.range",
      "limits": "Classification![2:2]"  
  },
  {
      "type": "filters.csf",
      "resolution": 0.25,
      "rigidness": 3,
      "iterations": 500,
      "threshold": 0.15,
  },

      
              {
                "type": "filters.hag_nn",
                "count": 100
              }
              ,
              {
                "type": "filters.range",
            "limits": "Classification![7:7],HeightAboveGround[0:]"
        },


    ]
}))
pipe.execute()
arrays = pipe.arrays[0]
            

In [6]:
voxel_resolution = (2, 2, 0.1)

voxels, extent = assign_voxels(arrays, voxel_resolution)
chm = calculate_chm(arrays, voxel_resolution, interpolation=None)
create_geotiff(chm[0],'/home/mgallet/Téléchargements/chm.tif', "EPSG:2154", extent)

In [ ]:
  
import pdal,json

pipe = pdal.Pipeline(json.dumps({
    "pipeline": [
        {
            "type": "readers.las",
            "filename": "/home/mgallet/Téléchargements/uncompress_LHD_FXX_0945_6473_PTS_O_LAMB93_IGN69.las",
            "spatialreference": "EPSG:2154",
            "start": 0,
            "count": 100000
        },
        # {
        # "type": "filters.voxeldownsize",
        # "cell": 0.2,
        # },
           {
            "type": "filters.outlier",
            "method": "statistical",
            "mean_k": 20,
            "multiplier": 2.0
        },
        {
            "type": "filters.csf",
            "resolution": 1.0,
            "iterations": 750,
            "rigidness": 2,
            "step": 0.75,
        },

        #         {
        #     "type": "filters.smrf",
        #     "slope": 0.25,
        #     "window": 20.0,
        #     "threshold": 0.2,
        #     "scalar": 1.25,
        #     "cell": 5.0,
        # },
        {
            "type": "filters.elm",
            "cell": 1.0,
            "class": 2,
            "threshold": 0.5
        },


        {
            "type": "filters.hag_nn",
            "count": 20,
        },
        {
            "type": "filters.range",
            "limits": "Classification![7:7],HeightAboveGround[0:5]"
        },
    ]
}))
pipe.execute()
arrays = pipe.arrays[0]

In [113]:
arrays.shape

(27888680,)

In [18]:
voxel_resolution = (2, 2, 0.1)
voxels, extent = assign_voxels(arrays, voxel_resolution)
chm = calculate_chm(arrays, voxel_resolution, interpolation=None)
create_geotiff(chm[0],'/home/mgallet/Téléchargements/chm.tif', "EPSG:2154", extent)

In [9]:
np.unique(arrays["Classification"], return_counts=True)

(array([2, 3, 4, 5], dtype=uint8), array([45533,   207,   202,   567]))

In [78]:
np.unique(arrays["Classification"], return_counts=True)

(array([1, 2, 3, 4, 5], dtype=uint8),
 array([ 3061, 44788,   192,   218,   511]))

In [10]:
arrays["HeightAboveGround"].max()

np.float64(12.597909963966686)

In [14]:
arrays["HeightAboveGround"]

array([-2.24604149,  0.        ,  0.86948369, ...,  0.        ,
        0.        ,  0.        ], shape=(1455,))

In [3]:
data = np.load(glob.glob(path2, recursive=True)[0])

In [4]:
data

array([(943090.8 , 6472468.32, 1605.82, 34362, 2, 0.),
       (943091.38, 6472468.26, 1606.16, 32789, 2, 0.),
       (943091.72, 6472468.22, 1606.32, 34733, 2, 0.), ...,
       (946211.31, 6473352.46, 2062.28, 52056, 2, 0.),
       (946083.77, 6473327.92, 1976.34, 51554, 2, 0.),
       (946113.11, 6473333.7 , 1996.48, 46311, 2, 0.)],
      shape=(18105435,), dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('Classification', 'u1'), ('HeightAboveGround', '<f8')])

In [ ]:
voxel_resolution = (20, 20, 0.1)
voxels, extent = assign_voxels(data, voxel_resolution)
chm = calculate_chm(data, voxel_resolution, interpolation=None)



In [6]:
create_geotiff(chm[0], "chm.tif", "EPSG:2154", extent)


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
from joblib import Parallel, delayed
from tqdm import tqdm

def compute_chm(points, resolution=1.0, quantile=100,  output_tif=None):
    # Extraire les colonnes
    x = points['X']
    y = points['Y']
    z = points['HeightAboveGround']  # ou points['Z'] si vous n'avez pas HeightAboveGround

    # Définir l’étendue
    xmin, xmax = np.min(x), np.max(x)
    ymin, ymax = np.min(y), np.max(y)

    width = int(np.ceil((xmax - xmin) / resolution))
    height = int(np.ceil((ymax - ymin) / resolution))

    # Convertir coordonnées XY en indices de grille
    col = ((x - xmin) / resolution).astype(int)
    row = ((ymax - y) / resolution).astype(int)  # attention à l'inversion Y

    # Grouper les points par cellule
    from collections import defaultdict
    grid_dict = defaultdict(list)
    for r, c, val in zip(row, col, z):
        grid_dict[(r, c)].append(val)

    # Fonction pour calculer le quantile sur une cellule
    def process_cell(cell):
        r, c = cell
        values = grid_dict[(r, c)]
        return (r, c, np.nanquantile(values, quantile / 100.0))

    # Parallélisation
    cells = list(grid_dict.keys())
    results = Parallel(n_jobs=-1)(delayed(process_cell)(cell) for cell in tqdm(cells))

    # Créer la grille CHM
    chm = np.full((height, width), np.nan, dtype=np.float32)
    for r, c, val in results:
        chm[r, c] = val

    # Export GeoTIFF
    if output_tif:
        transform = from_origin(xmin, ymax, resolution, resolution)
        with rasterio.open(
            output_tif, 'w',
            driver='GTiff',
            height=chm.shape[0],
            width=chm.shape[1],
            count=1,
            dtype=chm.dtype,
            crs='EPSG:2154',  # ou ajustez selon votre système de coordonnées
            transform=transform,
            nodata=np.nan
        ) as dst:
            dst.write(chm, 1)

    return chm

# Exemple d'utilisation
# chm = compute_chm(mon_array, resolution=1.0, quantile=95, classification_filter=[5], output_tif="chm_95.tif")


In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
from joblib import Parallel, delayed
from tqdm import tqdm

def compute_fhd(points, resolution=1.0, alti_resolution=1.0, output_tif=None):
    x = points['X']
    y = points['Y']
    z = points['HeightAboveGround']


    # Étendue spatiale
    xmin, xmax = np.min(x), np.max(x)
    ymin, ymax = np.min(y), np.max(y)
    width = int(np.ceil((xmax - xmin) / resolution))
    height = int(np.ceil((ymax - ymin) / resolution))

    # Indices grille
    col = ((x - xmin) / resolution).astype(int)
    row = ((ymax - y) / resolution).astype(int)

    # Groupement par cellule
    from collections import defaultdict
    grid_dict = defaultdict(list)
    for r, c, h in zip(row, col, z):
        grid_dict[(r, c)].append(h)

    # Fonction pour calculer la FHD sur une cellule
    def compute_cell_fhd(cell):
        r, c = cell
        heights = grid_dict[(r, c)]
        if len(heights) == 0:
            return (r, c, np.nan)
        # Binning
        bins = np.arange(0, max(heights)+alti_resolution, alti_resolution)
        hist, _ = np.histogram(heights, bins=bins)
        probs = hist / np.sum(hist)
        probs = probs[probs > 0]
        fhd = -np.sum(probs * np.log(probs))
        return (r, c, fhd)

    # Parallélisation
    cells = list(grid_dict.keys())
    results = Parallel(n_jobs=-1)(delayed(compute_cell_fhd)(cell) for cell in tqdm(cells))

    # Création raster FHD
    fhd_raster = np.full((height, width), np.nan, dtype=np.float32)
    for r, c, val in results:
        fhd_raster[r, c] = val

    # Export GeoTIFF
    if output_tif:
        transform = from_origin(xmin, ymax, resolution, resolution)
        with rasterio.open(
            output_tif, 'w',
            driver='GTiff',
            height=fhd_raster.shape[0],
            width=fhd_raster.shape[1],
            count=1,
            dtype=fhd_raster.dtype,
            crs='EPSG:2154',  # ajustez selon votre système de coordonnées
            transform=transform,
            nodata=np.nan
        ) as dst:
            dst.write(fhd_raster, 1)

    return fhd_raster

# Exemple d'utilisation
# fhd = compute_fhd(mon_array, resolution=1.0, classification_filter=[5], bin_size=1.0, output_tif="fhd.tif")


In [ ]:
import numpy as np
from joblib import Parallel, delayed
from collections import defaultdict
from tqdm import tqdm

def compute_chm_fhd_canal(points, resolution=1.0, classification_filter=[5], 
                          seuils_chm=[0.2, 2.0], bin_size=1.0):
    x = points['X']
    y = points['Y']
    z = points['HeightAboveGround']
    classification = points['Classification']

    # Filtrage
    mask = np.isin(classification, classification_filter)
    x, y, z = x[mask], y[mask], z[mask]

    # Dimensions grille
    xmin, xmax = np.min(x), np.max(x)
    ymin, ymax = np.min(y), np.max(y)
    width = int(np.ceil((xmax - xmin) / resolution))
    height = int(np.ceil((ymax - ymin) / resolution))

    # Index grille
    col = ((x - xmin) / resolution).astype(int)
    row = ((ymax - y) / resolution).astype(int)

    # Groupement par cellule
    grid_dict = defaultdict(list)
    for r, c, h in zip(row, col, z):
        grid_dict[(r, c)].append(h)

    # Calcul CHM et FHD pour chaque cellule
    def compute_cell_values(cell):
        r, c = cell
        heights = grid_dict[(r, c)]
        if not heights:
            return (r, c, np.nan, np.nan)

        heights = np.array(heights)
        chm = np.nanquantile(heights, 1.0)  # quantile 100%
        bins = np.arange(0, heights.max() + bin_size, bin_size)
        hist, _ = np.histogram(heights, bins=bins)
        probs = hist / np.sum(hist)
        probs = probs[probs > 0]
        fhd = -np.sum(probs * np.log(probs)) if len(probs) > 0 else 0.0

        return (r, c, chm, fhd)

    cells = list(grid_dict.keys())
    results = Parallel(n_jobs=-1)(delayed(compute_cell_values)(cell) for cell in tqdm(cells))

    # Matrices
    chm = np.full((height, width), np.nan, dtype=np.float32)
    fhd = np.full_like(chm, np.nan)
    canal = np.full_like(chm, 127, dtype=np.uint8)  # par défaut: nan

    # Remplissage
    seuil_bas, seuil_haut = seuils_chm
    for r, c, chm_val, fhd_val in results:
        chm[r, c] = chm_val
        fhd[r, c] = fhd_val

        if np.isnan(chm_val) or np.isnan(fhd_val):
            canal[r, c] = 127  # NaN
        elif chm_val == 0 and fhd_val == 0:
            canal[r, c] = 0
        elif 0 < chm_val <= seuil_bas:
            canal[r, c] = 1
        elif chm_val > seuil_haut:
            canal[r, c] = 2
        else:
            canal[r, c] = 0  # autres cas

    return canal


In [ ]:
voxels, extent = assign_voxels(data, voxel_resolution)
fhd = calculate_fhd(data, voxel_resolution)
create_geotiff(fhd, "fhd.tif", "EPSG:2154", extent)
del voxels

chm = calculate_chm(data, voxel_resolution, interpolation=None)
create_geotiff(chm, "chm.tif", "EPSG:2154", extent)

In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
from joblib import Parallel, delayed
from collections import defaultdict
from tqdm import tqdm

def compute_chm_fhd_and_canal(points, resolution=1.0, classification_filter=[5],
                               quantile=100, bin_size=1.0, seuils_chm=[0.2, 2.0]):
    x = points['X']
    y = points['Y']
    z = points['HeightAboveGround']
    classification = points['Classification']

    # Filtrage
    mask = np.isin(classification, classification_filter)
    x, y, z = x[mask], y[mask], z[mask]

    xmin, xmax = np.min(x), np.max(x)
    ymin, ymax = np.min(y), np.max(y)
    width = int(np.ceil((xmax - xmin) / resolution))
    height = int(np.ceil((ymax - ymin) / resolution))

    col = ((x - xmin) / resolution).astype(int)
    row = ((ymax - y) / resolution).astype(int)

    grid_dict = defaultdict(list)
    for r, c, h in zip(row, col, z):
        grid_dict[(r, c)].append(h)

    def process_cell(cell):
        r, c = cell
        heights = np.array(grid_dict[(r, c)])
        if len(heights) == 0:
            return (r, c, np.nan, np.nan)

        chm = np.nanquantile(heights, quantile / 100.0)
        bins = np.arange(0, heights.max() + bin_size, bin_size)
        hist, _ = np.histogram(heights, bins=bins)
        probs = hist / np.sum(hist)
        probs = probs[probs > 0]
        fhd = -np.sum(probs * np.log(probs)) if len(probs) > 0 else 0.0

        # Canal logique
        if np.isnan(chm) or np.isnan(fhd):
            canal = 127
        elif chm == 0 and fhd == 0:
            canal = 0
        elif 0 < chm <= seuils_chm[0]:
            canal = 1
        elif chm > seuils_chm[1]:
            canal = 2
        else:
            canal = 0

        return r, c, chm, fhd, canal

    cells = list(grid_dict.keys())
    results = Parallel(n_jobs=-1)(delayed(process_cell)(cell) for cell in tqdm(cells))

    # Matrices de sortie
    chm_img = np.full((height, width), np.nan, dtype=np.float32)
    fhd_img = np.full((height, width), np.nan, dtype=np.float32)
    canal_img = np.full((height, width), 127, dtype=np.uint8)

    for r, c, chm_val, fhd_val, canal_val in results:
        chm_img[r, c] = chm_val
        fhd_img[r, c] = fhd_val
        canal_img[r, c] = canal_val

    transform = from_origin(xmin, ymax, resolution, resolution)
    return chm_img, fhd_img, canal_img, transform, height, width


def export_3band_geotiff(filename, chm, fhd, canal, transform, crs='EPSG:2154'):
    with rasterio.open(
        filename, 'w',
        driver='GTiff',
        height=chm.shape[0],
        width=chm.shape[1],
        count=3,
        dtype=rasterio.float32,
        crs=crs,
        transform=transform,
        nodata=np.nan
    ) as dst:
        dst.write(chm.astype(np.float32), 1)
        dst.write(fhd.astype(np.float32), 2)
        dst.write(canal.astype(np.float32), 3)  # exporté en float32 pour cohérence multi-bandes


def compute_and_export_3band_image(points, output_tif,
                                   resolution=1.0,
                                   classification_filter=[5],
                                   quantile=100,
                                   bin_size=1.0,
                                   seuils_chm=[0.2, 2.0]):
    chm, fhd, canal, transform, height, width = compute_chm_fhd_and_canal(
        points=points,
        resolution=resolution,
        classification_filter=classification_filter,
        quantile=quantile,
        bin_size=bin_size,
        seuils_chm=seuils_chm
    )
    export_3band_geotiff(output_tif, chm, fhd, canal, transform)
    return chm, fhd, canal


chm, fhd, canal = compute_and_export_3band_image(
    points=mon_array,
    output_tif="output_3band_image.tif",
    resolution=1.0,
    classification_filter=[5],
    quantile=100,
    bin_size=1.0,
    seuils_chm=[0.2, 2.0]
)
